In [3]:
from basicTrainingData import y_test, X_test, tags, prey_train, preX_train, dict_vectorizer, add_basic_features, y_train, y_val, simple_train_sentences
from keras.models import load_model
import re
import pickle 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


#Deep learning model
training_model = load_model('keras_mlp.h5')
y_pred = training_model.model.predict(X_test, verbose=1).argmax(-1)
y_true = y_test.argmax(-1)

#Encoding model
pkl_file = open('Departure_encoder.pkl', 'rb')
encoder_model = pickle.load(pkl_file) 
pkl_file.close()



C:\Users\Precisit\Anaconda3\envs\NaturalLanguageToCode\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Precisit\Anaconda3\envs\NaturalLanguageToCode\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Precisit\Anaconda3\envs\NaturalLanguageToCode\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Precisi

Done

5061/5061 [==============================] - 3s 561us/step

In [4]:
counter = 0
for seq_index in range(10):
  print('-')
  print('Original input: ', preX_train[seq_index])
  print('Input sentence:', simple_train_sentences[seq_index])
  print('Decoded sentence:', prey_train[counter:counter+len(simple_train_sentences[seq_index])])
  counter += len(simple_train_sentences[seq_index])

-
Original input:  [('Without', 'IN'), ('the', 'DT'), ('Cray-3', 'NNP'), ('research', 'NN'), ('and', 'CC'), ('development', 'NN'), ('expenses', 'NNS'), (',', ','), ('the', 'DT'), ('company', 'NN'), ('would', 'MD'), ('have', 'VB'), ('been', 'VBN'), ('able', 'JJ'), ('*-2', '-NONE-'), ('to', 'TO'), ('report', 'VB'), ('a', 'DT'), ('profit', 'NN'), ('of', 'IN'), ('$', '$'), ('19.3', 'CD'), ('million', 'CD'), ('*U*', '-NONE-'), ('*ICH*-3', '-NONE-'), ('for', 'IN'), ('the', 'DT'), ('first', 'JJ'), ('half', 'DT'), ('of', 'IN'), ('1989', 'CD'), ('rather', 'RB'), ('than', 'IN'), ('the', 'DT'), ('$', '$'), ('5.9', 'CD'), ('million', 'CD'), ('*U*', '-NONE-'), ('0', '-NONE-'), ('it', 'PRP'), ('posted', 'VBD'), ('*T*-1', '-NONE-'), ('.', '.')]
Input sentence: ['Without', 'the', 'Cray-3', 'research', 'and', 'development', 'expenses', ',', 'the', 'company', 'would', 'have', 'been', 'able', '*-2', 'to', 'report', 'a', 'profit', 'of', '$', '19.3', 'million', '*U*', '*ICH*-3', 'for', 'the', 'first', 'hal

In [6]:
from nltk.corpus import wordnet
from collections import Counter

def get_part_of_speech(word):
  probable_part_of_speech = wordnet.synsets(word)
  pos_counts = Counter()
  pos_counts["n"] = len(  [ item for item in probable_part_of_speech if item.pos()=="n"]  )
  pos_counts["v"] = len(  [ item for item in probable_part_of_speech if item.pos()=="v"]  )
  pos_counts["a"] = len(  [ item for item in probable_part_of_speech if item.pos()=="a"]  )
  pos_counts["r"] = len(  [ item for item in probable_part_of_speech if item.pos()=="r"]  )
  
  most_likely_part_of_speech = pos_counts.most_common(1)[0][0]
  return most_likely_part_of_speech


In [7]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def clean_up_example(text):
    cleaned = re.sub('\W+', ' ', text)
    tokenized = word_tokenize(cleaned)
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(token, get_part_of_speech(token)) for token in tokenized]
    return lemmatized

#These properties could include informations about
# previous and next words as well as prefixes and suffixes.
def add_basic_features(sentence_terms, index):
    term = sentence_terms[index]
    return {
        'nb_terms': len(sentence_terms),
        'term': term,
        'is_first': index == 0,
        'is_last': index == len(sentence_terms) - 1,
        'is_capitalized': term[0].upper() == term[0],
        'is_all_caps': term.upper() == term,
        'is_all_lower': term.lower() == term,
        'prefix-1': term[0],
        'prefix-2': term[:2],
        'prefix-3': term[:3],
        'suffix-1': term[-1],
        'suffix-2': term[-2:],
        'suffix-3': term[-3:],
        'prev_word': '' if index == 0 else sentence_terms[index - 1],
        'next_word': '' if index == len(sentence_terms) - 1 else sentence_terms[index + 1]
    }


def give_tag(probabilities):
    #Take out index number of most probable in each list, then decodes them    
    maximum = probabilities.max()
    index_of_maximum = np.where(probabilities == maximum)
    tag = encoder_model.inverse_transform(index_of_maximum[0])
    return tag

In [11]:
import pickle
#What to keep?
import numpy as np

example = clean_up_example("walk down")
lem_list = []

for i in range(len(example)): 
    lem_list.append(add_basic_features(example, i))

example_result = training_model.predict(dict_vectorizer.transform(lem_list))

for i in range(len(lem_list)):
  print('-')
  print('Input word:', example[i])
  print('Decoded tag:',give_tag(example_result[i]))





-
Input word: walk
Decoded tag: ['VBP']
-
Input word: down
Decoded tag: ['RB']
